# 한빛아카데미 도서 크롤링

In [49]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
site = "https://www.hanbit.co.kr"
url = f"{site}/academy/books/new_book_list.html?page={1}&cate_cd=&srt=&searchKey=&keyWord="
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

## 1. 필요한 데이터 찾기

In [50]:
book_list = soup.select_one('.sub_book_list_area').find_all('li')

In [51]:
book = book_list[2]

In [52]:
new_url = f"{site}{book.select_one('.book_tit > a').attrs['href']}"
new_response = requests.get(new_url)
new_soup = BeautifulSoup(new_response.text, 'html.parser')

In [53]:
title = new_soup.select_one('.store_product_info_box').find('h3').getText().strip()
title

'리더십 : 이론과 실제 (9판)'

In [54]:
book_info = new_soup.select_one('.store_product_info_box').find_all('li')

In [55]:
author = book_info[0].find('span').getText().strip()
author

'Peter G. Northouse'

In [56]:
try:
    if (book_info[1].find('strong').getText() == '번역 : '):
        translator = book_info[1].find('span').getText().strip()
    else:
        translator = ""
except:
    translator = ""
translator

'김남현'

In [57]:
if (book_info[1].find('strong').getText() == '출간 : '):
    publish_date = book_info[1].find('span').getText().strip()

NameError: name 'publish_date' is not defined

In [58]:
if (book_info[2].find('strong').getText() == '페이지 : '):
    page_num = int(book_info[2].find('span').getText().split(' ')[0])

NameError: name 'page_num' is not defined

In [59]:
author, translator, publish_date, page_num = '','','',''
for li in book_info:
    if (li.find('strong').getText() == '저자 : '):
        author = li.find('span').getText().strip()
    try:
        if (li.find('strong').getText() == '번역 : '):
            translator = li.find('span').getText().strip()
    except:
        translator = ""
    if (li.find('strong').getText() == '출간 : '):
        publish_date = li.find('span').getText().strip()
    if (li.find('strong').getText() == '페이지 : '):
        page_num = int(li.find('span').getText().split(' ')[0])
    print(author, translator, publish_date, page_num)

Peter G. Northouse   
Peter G. Northouse 김남현  
Peter G. Northouse 김남현 2023-01-02 
Peter G. Northouse 김남현 2023-01-02 660
Peter G. Northouse 김남현 2023-01-02 660
Peter G. Northouse 김남현 2023-01-02 660


In [141]:
price = int(new_soup.select_one('.store_payment_area > fieldset').find_all('label')[-1].select('.pbr > strong')[0].getText().replace(',', ''))
price

30000

## 2. 리스트에 저장

In [63]:
lst = []
for i in range(1, 11):
    site = "https://www.hanbit.co.kr"
    url = f"{site}/academy/books/new_book_list.html?page={i}&cate_cd=&srt=&searchKey=&keyWord="
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser') 
    for book in tqdm(soup.select_one('.sub_book_list_area').find_all('li')):
        new_url = f"{site}{book.select_one('.book_tit > a').attrs['href']}"
        new_response = requests.get(new_url)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        
        book_info = new_soup.select_one('.store_product_info_box').find_all('li')        
        title = new_soup.select_one('.store_product_info_box').find('h3').getText().strip()
        for li in book_info:
            if (li.find('strong').getText() == '저자 : '):
                author = li.find('span').getText().strip()
            try:
                if (li.find('strong').getText() == '번역 : '):
                    translator = li.find('span').getText().strip()
            except:
                translator = ""
            if (li.find('strong').getText() == '출간 : '):
                publish_date = li.find('span').getText().strip()
            if (li.find('strong').getText() == '페이지 : '):
                page_num = int(li.find('span').getText().split(' ')[0])
        price = int(new_soup.select_one('.store_payment_area > fieldset').find_all('label')[-1].select('.pbr > strong')[0].getText().replace(',', ''))
        lst.append({
            '제목' : title, '저자' : author, '번역' : translator, '출간일' : publish_date, '쪽수' : page_num, '판매가' : price
        })
        translator = ""

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

## 3. 데이터 프레임으로 변환

In [64]:
df_hanbit = pd.DataFrame(lst)
df_hanbit

,제목,저자,번역,출간일,쪽수,판매가
0,"STEM CookBook, 전기 및 하이브리드 자동차(3판)",Iqbal Husain,"임원식 , 김정윤 , 김기찬 , 강동우",2023-01-10,592,35000
1,"IT CookBook, 컴퓨터 비전과 딥러닝",오일석,,2023-01-05,664,39000
2,리더십 : 이론과 실제 (9판),Peter G. Northouse,김남현,2023-01-02,660,36000
3,"IT CookBook, JAVA 마스터",송미영,,2023-01-01,732,30000
4,"IT CookBook, 데이터 과학을 위한 파이썬 프로그래밍(2판)",최성철,,2023-01-01,540,30000
...,...,...,...,...,...,...
195,NO BULLSHIT 선형대수학 가이드,Ivan Savov,"김대정 , 모현선 , 배준우",2020-01-06,628,28000
196,NO BULLSHIT 수학&물리 가이드,Ivan Savov,권기영,2020-01-06,568,28000
197,정보교과교육론(2판),"이태욱 , 최현종 , 전용주",,2020-01-06,464,27000
198,실무로 완성하는 PLC(멜섹Q) : 서보를 이용한 위치 제어,정완보,,2020-01-02,568,28000


## 4. csv 파일로 변환

In [67]:
df_hanbit.to_csv('한빛도서리스트.csv')

# 인터파크 도서 크롤링

In [37]:
import re

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
base_url = "http://book.interpark.com"
url = f"{base_url}/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001#"
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

## 1. 필요한 데이터 찾기

In [11]:
lis = soup.select('.rankBestContentList > ol > li')
lis

[<li>
 <!-- listItem | singleType or setType :: start -->
 <div class="listItem singleType">
 <div class="cover">
 <div class="coverImage">
 <label>
 <a href="/product/BookDisplay.do?_method=detail&amp;sc.shopNo=0000400000&amp;sc.prdNo=356179385&amp;sc.saNo=003003001&amp;bid1=Best_zone&amp;bid2=LiveRanking&amp;bid3=PRD&amp;bid4=001" target="_blank">
 <span class="checkBox">
 <input name="_KEY" type="checkbox" value="356179385"/>
 </span>
 <img alt="생에 감사해" src="https://bimage.interpark.com/partner/goods_image/9/3/8/5/356179385s.jpg"/>
 <i class="decoFront"></i>
 </a>
 </label>
 </div>
 <div class="coverOption">
 <ul>
 <li class="cart">
 <a class="cartGo" href="#" value="356179385"><span>CART</span></a>
 </li>
 <li class="buy">
 <a href="javascript:directOrd('356179385');"><span>BUY</span></a>
 </li>
 </ul>
 </div>
 <div class="rankNumber digit2">
 <span class="rankBtn_ctrl rkNum_B01"></span>
 </div>
 </div>
 <div class="itemIcon">
 <span class="bestSeller">베스트셀러</span>
 </div>
 <a href

In [13]:
li = lis[9]
href = li.select_one('.listItem').find('a')['href']
# href = li.select_one('.coverImage').find('a')['href']
href

'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=355179957&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [14]:
# 책제목
title = li.select_one('.itemName').getText().strip()
# 출판사
company = li.select_one('.company').get_text().strip()
# 가격 
price = int(li.select_one('.price > em').get_text().replace(',', ''))

In [16]:
# 1-9위
li.select_one('.rankBtn_ctrl')

<span class="rankBtn_ctrl rkNum_B08"></span>

In [17]:
li.select_one('.rankBtn_ctrl')['class']

['rankBtn_ctrl', 'rkNum_B08']

In [18]:
li.select_one('.rankBtn_ctrl')['class'][1]

'rkNum_B08'

In [19]:
li.select_one('.rankBtn_ctrl')['class'][1][-1]

'8'

In [20]:
# 10-15위
element = '<div class="rankNumber digit2"><span class="rankBtn_ctrl rkNum_M01 "></span><span class="rankBtn_ctrl rkNum_M02 "></span></div>'
mini_soup = BeautifulSoup(element, 'html.parser')
mini_soup

<div class="rankNumber digit2"><span class="rankBtn_ctrl rkNum_M01"></span><span class="rankBtn_ctrl rkNum_M02"></span></div>

In [23]:
mini_soup.select('.rankBtn_ctrl')

[<span class="rankBtn_ctrl rkNum_M01"></span>,
 <span class="rankBtn_ctrl rkNum_M02"></span>]

In [27]:
rank_data = mini_soup.select('.rankBtn_ctrl')
if len(rank_data) == 1:
    rank = int(rank_data[0]['class'][1][-1])
else:
    rank = rank_data[0]['class'][1][-1] + rank_data[1]['class'][1][-1]
    rank = int(rank)

In [28]:
rank

12

* sub page

In [29]:
url = base_url + href
res = requests.get(url)
sub_soup = BeautifulSoup(res.text, 'html.parser')

In [32]:
sub_lis = sub_soup.select('.bInfo_txt > li')
len(sub_lis)

5

In [34]:
for li in sub_lis:
    print(li.get_text())

저 : 정지아
출판사 : 창비
발행 : 2022년 09월 02일
쪽수 : 268
ISBN : 9788936438838


In [38]:
trans = '-'
for li in sub_lis:
    text = li.get_text().strip()
#     print(text)
    if '발행 :' in text:
        date = text.split(':')[1].strip()
        date = re.sub('[^0-9]', '', date)
    elif '쪽수 :' in text:
        page = int(text.split(':')[1].strip())
    elif '역 :' in text:
        trans = text.split(':')[1].strip()
    elif '저 :' in text:
        author = text.split(':')[1].strip()
print(date, page, trans, author)

20220902 268 - 정지아


## 2. 리스트에 저장

In [44]:
# tqdm 써서 전페이지 가져오기
lines = []
for li in tqdm(lis):
    href = li.select_one('.listItem').find('a')['href']
    # 책제목
    title = li.select_one('.itemName').getText().strip()
    # 출판사
    company = li.select_one('.company').get_text().strip()
    # 가격 
    price = int(li.select_one('.price > em').get_text().replace(',', ''))
    # 순위
    rank_data = li.select('.rankBtn_ctrl')
    if len(rank_data) == 1:
        rank = int(rank_data[0]['class'][1][-1])
    else:
        rank = rank_data[0]['class'][1][-1] + rank_data[1]['class'][1][-1]
        rank = int(rank)
    # sub page
    url = base_url + href
    res = requests.get(url)
    sub_soup = BeautifulSoup(res.text, 'html.parser')
    sub_lis = sub_soup.select('.bInfo_txt > li')
    trans = '-'
    for li in sub_lis:
        text = li.get_text().strip()
    #     print(text)
        if '발행 :' in text:
            date = text.split(':')[1].strip()
            date = re.sub('[^0-9]', '', date)
        elif '쪽수 :' in text:
            page = int(text.split(':')[1].strip())
        elif '역 :' in text:
            trans = text.split(':')[1].strip()
        elif '저 :' in text:
            author = text.split(':')[1].strip()
    lines.append([rank, title, author, trans, company, page, date, price])

  0%|          | 0/15 [00:00<?, ?it/s]

## 3. 데이터 프레임으로 변환

In [47]:
df_inter = pd.DataFrame(lines, columns=['순위', '제목', '저자', '역자', '출판사', '쪽수', '발행일', '가격'])
df_inter

,순위,제목,저자,역자,출판사,쪽수,발행일,가격
0,1,생에 감사해,김혜자,-,수오서재,376,20221222,15300
1,2,만일 내가 인생을 다시 산다면(10만 부 기념 스페셜 에디션),"김혜남, 김혜남",-,메이븐,280,20221111,15480
2,3,미스터 프레지던트,탁현민,-,메디치미디어,436,20230118,19800
3,4,안젤리크,기욤 뮈소,양영란,밝은세상,0,20221221,15120
4,5,원씽(The One Thing)(리커버 특별판),"게리 켈러, 제이 파파산",구세희,비즈니스북스,280,20130830,12600
5,6,스카이 버스,분당강쌤,-,다산에듀,280,20230103,16200
6,6,트렌드 코리아 2023,"김난도, 김난도, 추예린, 전미영, 전미영, 최지혜, 이수진, 권정윤, 이준영, 이...",-,미래의창,424,20221005,17100
7,8,빅 히스토리,"데이비드 크리스천, 신시아 브라운, 크레이그 벤저민",-,웅진지식하우스,640,20221223,29700
8,8,이토록 평범한 미래,김연수,-,문학동네,276,20221007,12600
9,8,아버지의 해방일지,정지아,-,창비,268,20220902,13500


## 4. csv 파일로 변환

In [68]:
df_inter.to_csv('인터파크도서리스트.csv')